# Constraints

Industrial boiler with startup costs, minimum uptime, and load constraints.

This notebook introduces:

- **StatusParameters**: Model on/off decisions with constraints
- **Startup costs**: Penalties for turning equipment on
- **Minimum uptime/downtime**: Prevent rapid cycling
- **Minimum load**: Equipment can't run below a certain output

## Setup

In [1]:
import pandas as pd
import xarray as xr

import flixopt as fx

fx.CONFIG.notebook()

flixopt.config.CONFIG

## System Description

The factory has:

- **Industrial boiler**: 500 kW capacity, startup cost of 50€, minimum 4h uptime
- **Small backup boiler**: 100 kW, no startup constraints (always available)
- **Steam demand**: Varies with production schedule (high during shifts, low overnight)

The main boiler is more efficient but has operational constraints. The backup is less efficient but flexible.

## Define Time Horizon and Demand

In [2]:
from data.tutorial_data import get_constraints_data

data = get_constraints_data()
timesteps = data['timesteps']
steam_demand = data['steam_demand']

In [3]:
# Visualize the demand with plotly
demand_ds = xr.Dataset(
    {
        'Steam Demand [kW]': xr.DataArray(steam_demand, dims=['time'], coords={'time': timesteps}),
    }
)
demand_ds.plotly.line(x='time', title='Factory Steam Demand')

## Build System with Operational Constraints

In [4]:
flow_system = fx.FlowSystem(timesteps, name='Constrained')

# Define and register custom carriers
flow_system.add_carriers(
    fx.Carrier('gas', '#3498db', 'kW'),
    fx.Carrier('steam', '#87CEEB', 'kW_th', 'Process steam'),
)

flow_system.add_elements(
    # === Buses ===
    fx.Bus('Gas', carrier='gas'),
    fx.Bus('Steam', carrier='steam'),
    # === Effect ===
    fx.Effect('costs', '€', 'Operating Costs', is_standard=True, is_objective=True),
    # === Gas Supply ===
    fx.Source(
        'GasGrid',
        outputs=[fx.Flow('Gas', bus='Gas', size=1000, effects_per_flow_hour=0.06)],
    ),
    # === Main Industrial Boiler (with operational constraints) ===
    fx.linear_converters.Boiler(
        'MainBoiler',
        thermal_efficiency=0.94,  # High efficiency
        # StatusParameters define on/off behavior
        status_parameters=fx.StatusParameters(
            effects_per_startup={'costs': 50},  # 50€ startup cost
            min_uptime=4,  # Must run at least 4 hours once started
            min_downtime=2,  # Must stay off at least 2 hours
        ),
        thermal_flow=fx.Flow(
            'Steam',
            bus='Steam',
            size=500,
            relative_minimum=0.3,  # Minimum load: 30% = 150 kW
        ),
        fuel_flow=fx.Flow('Gas', bus='Gas', size=600),  # Size required for status_parameters
    ),
    # === Backup Boiler (flexible, but less efficient) ===
    fx.linear_converters.Boiler(
        'BackupBoiler',
        thermal_efficiency=0.85,  # Lower efficiency
        # No status parameters = can turn on/off freely
        thermal_flow=fx.Flow('Steam', bus='Steam', size=150),
        fuel_flow=fx.Flow('Gas', bus='Gas'),
    ),
    # === Factory Steam Demand ===
    fx.Sink(
        'Factory',
        inputs=[fx.Flow('Steam', bus='Steam', size=1, fixed_relative_profile=steam_demand)],
    ),
)

## Run Optimization

In [5]:
flow_system.optimize(fx.solvers.HighsSolver(mip_gap=0.01));

## Analyze Results

### Steam Balance

See how the two boilers share the load:

In [6]:
flow_system.statistics.plot.balance('Steam')

PlotResult(data=<xarray.Dataset> Size: 2kB
Dimensions:              (time: 73)
Coordinates:
  * time                 (time) datetime64[ns] 584B 2024-03-11 ... 2024-03-14
Data variables:
    MainBoiler(Steam)    (time) float64 584B -0.0 -0.0 -0.0 ... -0.0 -0.0 nan
    BackupBoiler(Steam)  (time) float64 584B -58.29 -99.95 -85.66 ... -63.38 nan
    Factory(Steam)       (time) float64 584B 58.29 99.95 85.66 ... 63.38 nan, figure=Figure({
    'data': [{'fillcolor': '#EF553B',
              'fillpattern': {'shape': ''},
              'hovertemplate': '<b>%{fullData.name}</b>: %{y:.1f} kW_th<extra></extra>',
              'legendgroup': 'MainBoiler(Steam)',
              'line': {'color': '#EF553B', 'shape': 'hv', 'width': 0},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'MainBoiler(Steam)',
              'orientation': 'v',
              'showlegend': True,
              'stackgroup': 'negative',
              'type': 'scatter',
              'x': array(['2024-03-11T00:00:00.000000000', '2024-03-11T01:00:00.000000000',
                          '2024-03-11T02:00:00.000000000', '2024-03-11T03:00:00.000000000',
                          '2024-03-11T04:00:00.000000000', '2024-03-11T05:00:00.000000000',
                          '2024-03-11T06:00:00.000000000', '2024-03-11T07:00:00.000000000',
                          '2024-03-11T08:00:00.000000000', '2024-03-11T09:00:00.000000000',
                          '2024-03-11T10:00:00.000000000', '2024-03-11T11:00:00.000000000',
                          '2024-03-11T12:00:00.000000000', '2024-03-11T13:00:00.000000000',
                          '2024-03-11T14:00:00.000000000', '2024-03-11T15:00:00.000000000',
                          '2024-03-11T16:00:00.000000000', '2024-03-11T17:00:00.000000000',
                          '2024-03-11T18:00:00.000000000', '2024-03-11T19:00:00.000000000',
                          '2024-03-11T20:00:00.000000000', '2024-03-11T21:00:00.000000000',
                          '2024-03-11T22:00:00.000000000', '2024-03-11T23:00:00.000000000',
                          '2024-03-12T00:00:00.000000000', '2024-03-12T01:00:00.000000000',
                          '2024-03-12T02:00:00.000000000', '2024-03-12T03:00:00.000000000',
                          '2024-03-12T04:00:00.000000000', '2024-03-12T05:00:00.000000000',
                          '2024-03-12T06:00:00.000000000', '2024-03-12T07:00:00.000000000',
                          '2024-03-12T08:00:00.000000000', '2024-03-12T09:00:00.000000000',
                          '2024-03-12T10:00:00.000000000', '2024-03-12T11:00:00.000000000',
                          '2024-03-12T12:00:00.000000000', '2024-03-12T13:00:00.000000000',
                          '2024-03-12T14:00:00.000000000', '2024-03-12T15:00:00.000000000',
                          '2024-03-12T16:00:00.000000000', '2024-03-12T17:00:00.000000000',
                          '2024-03-12T18:00:00.000000000', '2024-03-12T19:00:00.000000000',
                          '2024-03-12T20:00:00.000000000', '2024-03-12T21:00:00.000000000',
                          '2024-03-12T22:00:00.000000000', '2024-03-12T23:00:00.000000000',
                          '2024-03-13T00:00:00.000000000', '2024-03-13T01:00:00.000000000',
                          '2024-03-13T02:00:00.000000000', '2024-03-13T03:00:00.000000000',
                          '2024-03-13T04:00:00.000000000', '2024-03-13T05:00:00.000000000',
                          '2024-03-13T06:00:00.000000000', '2024-03-13T07:00:00.000000000',
                          '2024-03-13T08:00:00.000000000', '2024-03-13T09:00:00.000000000',
                          '2024-03-13T10:00:00.000000000', '2024-03-13T11:00:00.000000000',
                          '2024-03-13T12:00:00.000000000', '2024-03-13T13:00:00.000000000',
                          '2024-03-13T14:00:00.000000000', '2024-03-13T15:00:00.000000000',
                          '2024-03-13T16:00:00.000000000', '2024

### Main Boiler Operation

Notice how the main boiler:
- Runs continuously during production (respecting min uptime)
- Stays above minimum load (30%)
- Shuts down during low-demand periods

In [7]:
flow_system.statistics.plot.heatmap('MainBoiler(Steam)')

PlotResult(data=<xarray.Dataset> Size: 992B
Dimensions:    (timeframe: 4, timestep: 24)
Coordinates:
  * timeframe  (timeframe) object 32B '2024-03-11' '2024-03-12' ... '2024-03-14'
  * timestep   (timestep) object 192B '00:00' '01:00' ... '22:00' '23:00'
Data variables:
    value      (timestep, timeframe) float64 768B 0.0 0.0 0.0 ... 0.0 0.0 nan, figure=Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': ('timeframe: %{x}<br>timestep: %' ... 'flow_rate: %{z}<extra></extra>'),
              'name': '0',
              'type': 'heatmap',
              'x': array(['2024-03-11', '2024-03-12', '2024-03-13', '2024-03-14'], dtype=object),
              'xaxis': 'x',
              'y': array(['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00',
                          '08:00', '09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00',
                          '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00', '23:00'],
                         dtype=object),
              'yaxis': 'y',
              'z': {'bdata': ('AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA' ... 'AAAAAAAAAAAAAAAAAAAAAAAAAAAPh/'),
                    'dtype': 'f8',
                    'shape': '24, 4'}}],
    'layout': {'coloraxis': {'cmax': 435.0977236396222,
                             'cmin': 0.0,
                             'colorbar': {'title': {'text': 'MainBoiler(Steam)|flow_rate'}},
                             'colorscale': [[0.0, '#30123b'],
                                            [0.07142857142857142, '#4145ab'],
                                            [0.14285714285714285, '#4675ed'],
                                            [0.21428571428571427, '#39a2fc'],
                                            [0.2857142857142857, '#1bcfd4'],
                                            [0.35714285714285715, '#24eca6'],
                                            [0.42857142857142855, '#61fc6c'], [0.5,
                                            '#a4fc3b'], [0.5714285714285714,
                                            '#d1e834'], [0.6428571428571429,
                                            '#f3c63a'], [0.7142857142857143,
                                            '#fe9b2d'], [0.7857142857142857,
                                            '#f36315'], [0.8571428571428571,
                                            '#d93806'], [0.9285714285714286,
                                            '#b11901'], [1.0, '#7a0402']]},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'timeframe'}},
               'yaxis': {'anchor': 'x', 'autorange': 'reversed', 'domain': [0.0, 1.0], 'title': {'text': 'timestep'}}}
}))

### On/Off Status

Track the boiler's operational status:

In [8]:
# Merge solution DataArrays directly - xarray aligns coordinates automatically
status_ds = xr.Dataset(
    {
        'Status': flow_system.solution['MainBoiler|status'],
        'Steam Production [kW]': flow_system.solution['MainBoiler(Steam)|flow_rate'],
    }
)

status_ds.plotly.line(x='time', title='Main Boiler Operation', height=300)

### Startup Count and Costs

In [9]:
total_startups = int(flow_system.solution['MainBoiler|startup'].sum().item())
total_costs = flow_system.solution['costs'].item()
startup_costs = total_startups * 50
gas_costs = total_costs - startup_costs

pd.DataFrame(
    {
        'Startups': {'Count': total_startups, 'EUR': startup_costs},
        'Gas': {'Count': '-', 'EUR': gas_costs},
        'Total': {'Count': '-', 'EUR': total_costs},
    }
)

,Startups,Gas,Total
Count,3,-,-
EUR,150,1291.320174,1441.320174


### Duration Curves

See how often each boiler operates at different load levels:

In [10]:
flow_system.statistics.plot.duration_curve('MainBoiler(Steam)')

PlotResult(data=<xarray.Dataset> Size: 1kB
Dimensions:            (duration: 73)
Coordinates:
  * duration           (duration) int64 584B 0 1 2 3 4 5 6 ... 67 68 69 70 71 72
Data variables:
    MainBoiler(Steam)  (duration) float64 584B nan 435.1 429.9 ... 0.0 0.0 0.0, figure=Figure({
    'data': [{'hovertemplate': 'variable=MainBoiler(Steam)<br>duration=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'MainBoiler(Steam)',
              'line': {'color': '#636EFA', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'MainBoiler(Steam)',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': {'bdata': ('AAECAwQFBgcICQoLDA0ODxAREhMUFR' ... 'Q1Njc4OTo7PD0+P0BBQkNERUZHSA=='),
                    'dtype': 'i1'},
              'xaxis': 'x',
              'y': {'bdata': ('AAAAAAAA+H+Tw6lGkDF7QCDE7iqb3n' ... 'AAAAAAAAAAAAAAAAAAAAAAAAAAAAA='),
                    'dtype': 'f8'},
              'yaxis': 'y'}],
    'layout': {'legend': {'title': {'text': 'variable'}, 'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Duration Curve [kW_th]'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Timesteps'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'value'}}}
}))

In [11]:
flow_system.statistics.plot.duration_curve('BackupBoiler(Steam)')

PlotResult(data=<xarray.Dataset> Size: 1kB
Dimensions:              (duration: 73)
Coordinates:
  * duration             (duration) int64 584B 0 1 2 3 4 5 ... 67 68 69 70 71 72
Data variables:
    BackupBoiler(Steam)  (duration) float64 584B nan 127.8 ... -2.543e-14, figure=Figure({
    'data': [{'hovertemplate': 'variable=BackupBoiler(Steam)<br>duration=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'BackupBoiler(Steam)',
              'line': {'color': '#636EFA', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'BackupBoiler(Steam)',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': {'bdata': ('AAECAwQFBgcICQoLDA0ODxAREhMUFR' ... 'Q1Njc4OTo7PD0+P0BBQkNERUZHSA=='),
                    'dtype': 'i1'},
              'xaxis': 'x',
              'y': {'bdata': ('AAAAAAAA+H/DcRZAOvZfQHoLJc/SMF' ... 'nA0x0XvXh0E0qVJBy9UQm+McCgHL0='),
                    'dtype': 'f8'},
              'yaxis': 'y'}],
    'layout': {'legend': {'title': {'text': 'variable'}, 'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Duration Curve [kW_th]'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Timesteps'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'value'}}}
}))

## Compare: Without Operational Constraints

What if the main boiler had no startup costs or minimum uptime?

In [12]:
# Build unconstrained system
fs_unconstrained = fx.FlowSystem(timesteps, name='Unconstrained')
fs_unconstrained.add_carriers(
    fx.Carrier('gas', '#3498db', 'kW'),
    fx.Carrier('steam', '#87CEEB', 'kW_th', 'Process steam'),
)

fs_unconstrained.add_elements(
    fx.Bus('Gas', carrier='gas'),
    fx.Bus('Steam', carrier='steam'),
    fx.Effect('costs', '€', 'Operating Costs', is_standard=True, is_objective=True),
    fx.Source('GasGrid', outputs=[fx.Flow('Gas', bus='Gas', size=1000, effects_per_flow_hour=0.06)]),
    # Main boiler WITHOUT status parameters
    fx.linear_converters.Boiler(
        'MainBoiler',
        thermal_efficiency=0.94,
        thermal_flow=fx.Flow('Steam', bus='Steam', size=500),
        fuel_flow=fx.Flow('Gas', bus='Gas'),
    ),
    fx.linear_converters.Boiler(
        'BackupBoiler',
        thermal_efficiency=0.85,
        thermal_flow=fx.Flow('Steam', bus='Steam', size=150),
        fuel_flow=fx.Flow('Gas', bus='Gas'),
    ),
    fx.Sink('Factory', inputs=[fx.Flow('Steam', bus='Steam', size=1, fixed_relative_profile=steam_demand)]),
)

fs_unconstrained.optimize(fx.solvers.HighsSolver())
unconstrained_costs = fs_unconstrained.solution['costs'].item()

pd.DataFrame(
    {
        'Without Constraints': {'Cost [EUR]': unconstrained_costs},
        'With Constraints': {'Cost [EUR]': total_costs},
        'Overhead': {
            'Cost [EUR]': total_costs - unconstrained_costs,
            '%': (total_costs - unconstrained_costs) / unconstrained_costs * 100,
        },
    }
)

,Without Constraints,With Constraints,Overhead
Cost [EUR],1278.227071,1441.320174,163.093102
%,NaN,NaN,12.759322


### Side-by-Side Comparison

Use the `Comparison` class to visualize both systems together:

In [13]:
comp = fx.Comparison([fs_unconstrained, flow_system])
comp.statistics.plot.effects()

PlotResult(data=<xarray.Dataset> Size: 48B
Dimensions:  (case: 2)
Coordinates:
  * case     (case) object 16B 'Unconstrained' 'Constrained'
Data variables:
    costs    (case) float64 16B 1.278e+03 1.441e+03
    Penalty  (case) float64 16B 0.0 0.0, figure=Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'case=Unconstrained<br>variable=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Unconstrained',
              'marker': {'color': '#636EFA', 'line': {'width': 0}, 'pattern': {'shape': ''}},
              'name': 'Unconstrained',
              'offsetgroup': 'Unconstrained',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['costs', 'Penalty'], dtype=object),
              'xaxis': 'x',
              'y': {'bdata': 'icFbhej4k0AAAAAAAAAAAA==', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': 'case=Constrained<br>variable=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Constrained',
              'marker': {'color': '#EF553B', 'line': {'width': 0}, 'pattern': {'shape': ''}},
              'name': 'Constrained',
              'offsetgroup': 'Constrained',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['costs', 'Penalty'], dtype=object),
              'xaxis': 'x',
              'y': {'bdata': 'buWP20eFlkAAAAAAAAAAAA==', 'dtype': 'f8'},
              'yaxis': 'y'}],
    'layout': {'bargap': 0,
               'bargroupgap': 0,
               'barmode': 'group',
               'legend': {'title': {'text': 'case'}, 'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Effects Comparison (total)'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'variable'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'value'}}}
}))

### Energy Flow Sankey

A Sankey diagram visualizes the total energy flows through the system:

In [14]:
flow_system.statistics.plot.sankey.flows()

PlotResult(data=<xarray.Dataset> Size: 1kB
Dimensions:  (link: 6)
Coordinates:
  * link     (link) int64 48B 0 1 2 3 4 5
    source   (link) <U12 288B 'Gas' 'BackupBoiler' ... 'Gas' 'MainBoiler'
    target   (link) <U12 288B 'BackupBoiler' 'Steam' ... 'MainBoiler' 'Steam'
    label    (link) <U19 456B 'BackupBoiler(Gas)' ... 'MainBoiler(Steam)'
    carrier  (link) <U5 120B 'gas' 'steam' 'steam' 'gas' 'gas' 'steam'
Data variables:
    value    (link) float64 48B 2.279e+03 1.937e+03 ... 1.924e+04 1.809e+04, figure=Figure({
    'data': [{'link': {'color': [rgba(52, 152, 219, 0.4), rgba(135, 206, 235,
                                 0.4), rgba(135, 206, 235, 0.4), rgba(52, 152, 219,
                                 0.4), rgba(52, 152, 219, 0.4), rgba(135, 206, 235,
                                 0.4)],
                       'label': [BackupBoiler(Gas), BackupBoiler(Steam),
                                 Factory(Steam), GasGrid(Gas), MainBoiler(Gas),
                                 MainBoiler(Steam)],
                       'source': [1, 4, 2, 0, 1, 5],
                       'target': [4, 2, 3, 1, 5, 2],
                       'value': [2279.1696558721587, 1937.2942074913349,
                                 20025.557449144777, 21522.002891673696,
                                 19242.833235801536, 18088.26324165344]},
              'node': {'color': [#636EFA, #3498db, #87CEEB, #AB63FA, #00CC96,
                                 #EF553B],
                       'label': [GasGrid, Gas, Steam, Factory, BackupBoiler,
                                 MainBoiler],
                       'line': {'color': 'black', 'width': 0.5},
                       'pad': 15,
                       'thickness': 20},
              'type': 'sankey'}],
    'layout': {'template': '...', 'title': {'text': 'Energy Flow'}}
}))

## Key Concepts

### StatusParameters Options

```python
fx.StatusParameters(
    # Startup/shutdown costs
    effects_per_startup={'costs': 50},     # Cost per startup event
    effects_per_shutdown={'costs': 10},    # Cost per shutdown event
    
    # Time constraints
    min_uptime=4,       # Minimum hours running once started
    min_downtime=2,     # Minimum hours off once stopped
    
    # Startup limits
    max_startups=10,    # Maximum startups per period
)
```

### Minimum Load

Set via `Flow.relative_minimum`:
```python
fx.Flow('Steam', bus='Steam', size=500, relative_minimum=0.3)  # Min 30% load
```

### When Status is Active

- When `StatusParameters` is set, a binary on/off variable is created
- Flow is zero when status=0, within bounds when status=1
- Without `StatusParameters`, flow can vary continuously from 0 to max

## Summary

You learned how to:

- Add **startup costs** with `effects_per_startup`
- Set **minimum run times** with `min_uptime` and `min_downtime`
- Define **minimum load** with `relative_minimum`
- Access **status variables** from the solution
- Use **duration curves** to analyze operation patterns

### Next Steps

- **[05-multi-carrier-system](05-multi-carrier-system.ipynb)**: Model CHP with electricity and heat
- **[06a-time-varying-parameters](06a-time-varying-parameters.ipynb)**: Variable efficiency based on external conditions